In [ ]:
# Add src to path
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Added {src_path} to sys.path")

## Setup Cameras with Digital Twin

In [ ]:
from telescope_mcp.drivers.cameras import DigitalTwinCameraDriver
from telescope_mcp.devices import CameraRegistry, CameraConfig

# Create driver
driver = DigitalTwinCameraDriver()

# Create registry and get cameras
registry = CameraRegistry(driver)

# Discover cameras
cameras = registry.discover()
print(f"Found {len(cameras)} cameras:")
for cam_id, info in cameras.items():
    print(f"  {cam_id}: {info.name}")

# Get cameras (assuming 0=finder, 1=main based on twin defaults)
finder = registry.get(0)
main = registry.get(1)

# Connect both
finder.connect()
main.connect()

print(f"\nFinder connected: {finder.is_connected}")
print(f"Main connected: {main.is_connected}")

## Test Timing Calculation

In [ ]:
from telescope_mcp.devices import CameraController

controller = CameraController({
    "finder": finder,
    "main": main,
})

# Calculate delay for typical alignment scenario:
# Finder: 176 seconds long exposure
# Main: 312 ms short exposure
primary_us = 176_000_000  # 176 seconds
secondary_us = 312_000     # 312 ms

delay_us = controller.calculate_sync_timing(primary_us, secondary_us)
delay_sec = delay_us / 1_000_000

print(f"Primary (finder): {primary_us / 1_000_000:.1f} seconds")
print(f"Secondary (main): {secondary_us / 1000:.1f} ms")
print(f"\nDelay before starting secondary: {delay_sec:.3f} seconds")
print(f"This centers the {secondary_us/1000:.0f}ms exposure at the {primary_us/2/1_000_000:.1f}s midpoint")

## Test Short Synchronized Capture

Use short exposures for fast testing (0.5s primary, 0.1s secondary):

In [ ]:
from telescope_mcp.devices import SyncCaptureConfig
import time

# Short test exposures
config = SyncCaptureConfig(
    primary="finder",
    secondary="main",
    primary_exposure_us=500_000,    # 0.5 seconds
    secondary_exposure_us=100_000,  # 0.1 seconds
)

print("Starting synchronized capture...")
start = time.time()

result = controller.sync_capture(config)

elapsed = time.time() - start
print(f"Completed in {elapsed:.3f} seconds\n")

# Show results
print("Timing Analysis:")
print(f"  Ideal secondary start: {result.ideal_secondary_start_us / 1000:.1f} ms after primary")
print(f"  Actual secondary start: {result.actual_secondary_start_us / 1000:.1f} ms after primary")
print(f"  Timing error: {result.timing_error_ms:.2f} ms")

print(f"\nCaptured frames:")
print(f"  Primary: {len(result.primary_frame.image_data)} bytes at {result.primary_start}")
print(f"  Secondary: {len(result.secondary_frame.image_data)} bytes at {result.secondary_start}")

# Check timing accuracy (should be < 50ms typically)
if abs(result.timing_error_ms) < 50:
    print(f"\n✅ Timing accuracy is good ({result.timing_error_ms:.2f}ms error)")
else:
    print(f"\n⚠️  Timing error is higher than expected ({result.timing_error_ms:.2f}ms)")

## Test Realistic Alignment Scenario

**Note:** This will take ~3 minutes to complete (176 second primary exposure).

In [ ]:
# Uncomment to test realistic alignment timing

# config = SyncCaptureConfig(
#     primary="finder",
#     secondary="main",
#     primary_exposure_us=176_000_000,  # 176 seconds
#     secondary_exposure_us=312_000,     # 312 ms
# )

# print("Starting 176-second synchronized capture...")
# print("(Secondary will start at ~88 seconds)\n")

# import time
# start = time.time()

# result = controller.sync_capture(config)

# elapsed = time.time() - start
# print(f"\nCompleted in {elapsed:.1f} seconds")

# print("\nTiming Analysis:")
# print(f"  Ideal secondary start: {result.ideal_secondary_start_us / 1_000_000:.3f} s after primary")
# print(f"  Actual secondary start: {result.actual_secondary_start_us / 1_000_000:.3f} s after primary")
# print(f"  Timing error: {result.timing_error_ms:.2f} ms")

## Test with Custom Clock (for Testing)

Verify timing calculation works with injectable clock:

In [ ]:
class FakeClock:
    """Clock that records sleep calls without actually sleeping."""
    def __init__(self):
        self._time = 0.0
        self.sleeps = []
    
    def monotonic(self) -> float:
        return self._time
    
    def sleep(self, seconds: float) -> None:
        self.sleeps.append(seconds)
        self._time += seconds

# Create controller with fake clock
fake_clock = FakeClock()
test_controller = CameraController(
    cameras={"finder": finder, "main": main},
    clock=fake_clock,
)

# Run sync capture
config = SyncCaptureConfig(
    primary="finder",
    secondary="main",
    primary_exposure_us=176_000_000,
    secondary_exposure_us=312_000,
)

result = test_controller.sync_capture(config)

# Check that sleep was called with correct delay
expected_delay = (176_000_000 / 2 - 312_000 / 2) / 1_000_000
print(f"Expected delay: {expected_delay:.3f} seconds")
print(f"Actual sleep calls: {fake_clock.sleeps}")
print(f"\n✅ Fake clock test passed!" if fake_clock.sleeps else "⚠️ No sleep calls recorded")

## Cleanup

In [ ]:
# Disconnect cameras
finder.disconnect()
main.disconnect()

print("Cameras disconnected")